In [ ]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline 

In [277]:
# load normalized data and true labels
scaled_data = pd.read_csv('../data/GSE910_scaled_top_100.csv', index_col=0)
top_vars = pd.read_csv('../data/_GSE910_top_100_var.csv')
labels = pd.read_csv('../data/GSE910_labels.csv',index_col=0)

# encode categorical labels to numerical codes
labels = labels['Response'].values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
print("Label Map:")
for encoded_value, original_label in enumerate(label_encoder.classes_):
    print(f"{encoded_value}: {original_label}")

# count labels
label_counts = {}

for label in labels:
    label_counts[label] = label_counts.get(label, 0) + 1

for label, count in label_counts.items():
    print(f"Label '{label}': {count}")



Label Map:
0: N
1: R
Label 'N': 96
Label 'R': 54


In [ ]:
# fix class imbalance

# use SMOTE 
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(scaled_data, labels_encoded)

# count labels after resampling
resampled_counts = {}

for label in y_resampled:
    label_counts[label] = label_counts.get(label, 0) + 1

for label, count in label_counts.items():
    print(f"Label '{label}': {count}")

print(X_resampled)



In [ ]:
# fit PCA to data
pca = PCA(n_components=10)
pca.fit_transform(X_resampled)

In [ ]:
# use elbow method on variance explained to select optimal number of principal components
explained_variance = pca.explained_variance_ratio_

# plot explained variance against principal component
plt.plot(range(1, len(explained_variance) + 1), pca.explained_variance_ratio_, marker='o')
plt.xlabel('Principal Component')
plt.xticks(np.arange(1, len(pca.explained_variance_ratio_) + 1, 1))
plt.ylabel('Explained Variance')
plt.title('Explained Variance by Principal Components')
plt.show()

# also check cumulative variance explained
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.title('Cumulative Variance Explained by Principle Components')
plt.xlabel('Number of Principal Components')
plt.xticks(np.arange(1, len(pca.explained_variance_ratio_) + 1, 1))
plt.ylabel('Cumulative Variance Explained') ;

In [ ]:
# 2 principal components seems to be optimal
pca2 = PCA(n_components=6)
PCAreduced_data = pca2.fit_transform(X_resampled)

In [ ]:
# Get the loadings (coefficients) for the first two principal components
loadings = pca2.components_[:2]
# Get the absolute values of the loadings
# abs_loadings = np.abs(loadings)
# Find the indices of top features for each principal component in descending order
top_features_indices = np.argsort(loadings, axis=1)[:, ::-1]
# Make the column headers in scaled_data as a list
genes = list(scaled_data.columns)
print(genes)
# Get the names of features corresponding to the top indices
top_features = [genes[idx] for idx in top_features_indices.flatten()]
print(top_features)

In [ ]:
# To find the gene names of the top contributing genes of the PCA
top = pd.read_csv('../data/GSE910_top_100_var.csv')
Gene1 = top.iloc[91, 0]
print('Gene1:', Gene1)
Gene2 = top.iloc[67, 0]
print('Gene2:',Gene2)

In [ ]:
plt.scatter(PCAreduced_data[:, 0], PCAreduced_data[:, 1], c=y_resampled)
plt.xlabel('First principal component')
plt.ylabel('Second principal component')
plt.legend()
plt.show()

In [ ]:
# Train a logistic regression model using the top PCAs
logreg = sklearn.linear_model.LogisticRegression()
logreg.fit(PCAreduced_data,y_resampled)
y_pred = logreg.predict(PCAreduced_data)
acc = accuracy_score(y_resampled, y_pred)
print("Accuracy score of LogReg model on top PCAs:", acc)

In [ ]:
# Train a logistic regression model using the top 2 PCAs before optimizing hyperparameters
X_train, X_test, y_train, y_test = train_test_split(PCAreduced_data, y_resampled, test_size = 0.3, random_state=1210)
logreg = sklearn.linear_model.LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_train)
training_accuracy = accuracy_score(y_pred, y_train)
y_pred = logreg.predict(X_test)
test_accuracy = accuracy_score(y_pred, y_test)
print("Training accuracy before hyperparameter optimization:", training_accuracy)
print("Test set accuracy before hyperparameter optimization:", test_accuracy)

In [ ]:
# Train a logistic regression model using the top 2 PCAs
logreg = sklearn.linear_model.LogisticRegression(penalty="l1", C=0.1,solver='liblinear')

logreg.fit(PCAreduced_data,y_resampled)
y_pred = logreg.predict(PCAreduced_data)
acc = accuracy_score(y_resampled, y_pred)
print("Accuracy score of LogReg model on top 2 PCAs:", acc)


In [ ]:
# find the best hyperparameters for an L1 regularized LogReg model
hyperparameters = {
    'C': [0.1, 1, 10, 100, 1000],
    'solver': ['liblinear', 'saga'],
    'max_iter': [1000]
}

grid_search = GridSearchCV(LogisticRegression(penalty="l1"), hyperparameters,  cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best hyperparameters:", best_params)

In [ ]:
# Train a logistic regression model using the top 2 PCAs after optimizing hyperparameters
y_pred = best_model.predict(X_train)
training_accuracy = accuracy_score(y_pred, y_train)
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_pred, y_test)

print("Training accuracy after hyperparameter optimization:", training_accuracy)
print("Test set accuracy after hyperparameter optimization:", test_accuracy)

In [ ]:
# cv of PCA (number of PCs) and L1 logistic regression

pipeline = Pipeline([
    ('pca', PCA()),
    ('clf', LogisticRegression(penalty = 'l1'))

])

hyperparameters = {
    'pca__n_components': list(range(1, 16)),
    'clf__C': [0.1, 1, 10, 100, 1000],
    'clf__solver': ['liblinear', 'saga'],
    'clf__max_iter': [1000]
}

grid_search = GridSearchCV(pipeline, hyperparameters,  cv=5, scoring = 'accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best hyperparameters:", best_params)